In [2]:
import csv
import os
import sys
import pandas as pd
import numpy as np

In [3]:
#read the csv file
def read_csv(file):
    data = pd.read_csv(file)
    return data


In [4]:
file = "TV Series.csv"
df = read_csv(file)
print(df.head())

      Series Title Release Year Runtime                    Genre Rating  \
0        Wednesday     (2022– )  45 min   Comedy, Crime, Fantasy    8.2   
1      Yellowstone     (2018– )  60 min           Drama, Western    8.7   
2  The White Lotus  (2021–2023)  60 min            Comedy, Drama    7.9   
3             1923  (2022–2023)  60 min           Drama, Western    8.6   
4        Jack Ryan     (2018– )  60 min  Action, Drama, Thriller    8.0   

                                                Cast  \
0  Jenna Ortega, Hunter Doohan, Percy Hynes White...   
1  Kevin Costner, Luke Grimes, Kelly Reilly, Wes ...   
2  Jennifer Coolidge, Jon Gries, F. Murray Abraha...   
3  Harrison Ford, Helen Mirren, Brandon Sklenar, ...   
4  John Krasinski, Wendell Pierce, Michael Kelly,...   

                                            Synopsis  
0  Follows Wednesday Addams' years as a student, ...  
1  A ranching family in Montana faces off against...  
2  Set in a tropical resort, it follows the exp

In [5]:
df.shape

(50000, 7)

In [6]:
df.sample(10)

,Series Title,Release Year,Runtime,Genre,Rating,Cast,Synopsis
13097,The Lord of the Rings: The Rings of Power,(2022– ),****,"Action, Adventure, Drama",6.9,"Morfydd Clark, Ismael Cruz Cordova, Charlie Vi...",Epic drama set thousands of years before the e...
5495,Selling the OC,(2022– ),30 min,Reality-TV,5.5,"Alexandra Jarvis, Tyler Stanaland, Alexandra R...","Jason Oppenheim, owner of the Oppenheim Group,..."
47010,Ginny & Georgia,(2021– ),****,"Comedy, Drama",7.4,"Brianne Howey, Antonia Gentry, Diesel La Torra...","Ginny Miller, an angsty fifteen-year-old, ofte..."
5542,The Surreal Life,(2003– ),22 min,"Comedy, Reality-TV",5.4,"Charo, Dave Coulier, Flavor Flav, Jordan Knight",A reality show about fading celebrities who we...
22225,Criminal Minds,(2005– ),42 min,"Crime, Drama, Mystery",8.1,"Kirsten Vangsness, Matthew Gray Gubler, A.J. C...",A group of criminal profilers who work for the...
9834,Irene Huss,(2007–2011),90 min,"Crime, Drama, Thriller",6.6,"Angela Kovacs, Reuben Sallmander, Lars Brandeb...",Police investigator Irene Huss lives in Gothen...
18877,Better Call Saul,(2015–2022),46 min,"Crime, Drama",8.9,"Bob Odenkirk, Rhea Seehorn, Jonathan Banks, Pa...",The trials and tribulations of criminal lawyer...
32035,Hunters,(2020–2023),60 min,"Crime, Drama, Mystery",7.2,"Al Pacino, Logan Lerman, Lena Olin, Jerrika Hi...","In 1977 New York City, a troubled young Jewish..."
20502,The White Lotus,(2021–2023),60 min,"Comedy, Drama",7.9,"Jennifer Coolidge, Jon Gries, F. Murray Abraha...","Set in a tropical resort, it follows the explo..."
1051,Çukur,(2017–2021),120 min,"Action, Crime, Thriller",7.5,"Aras Bulut Iynemli, Erkan Kolçak Köstendil, Ri...","A dangerous neighborhood, ""The Pit"", ran by a ..."


In [7]:
def tweak_imdb_df(df: pd.DataFrame) -> pd.DataFrame:
    return (
        df
        # Formatting column names (Series Title -> series_title)
        .rename(columns=lambda column_: column_.lower().replace(" ", "_"))
        # Replacing "****" values, converting column type and extracting relevant info
        .assign(rating=lambda df_: df_.rating.replace("****", np.NaN).astype("float32"),
                runtime=lambda df_: df_.runtime.replace("****", np.NaN).str.extract(r"(\d+) min").astype("float").astype("Int64"),
                end_year=lambda df_: df_.release_year.str.extract(r"-?(\d+)\)").astype("float").astype("Int64"),
                release_year=lambda df_: df_.release_year.str.extract(r"\((\d+)–?").astype("float").astype("Int64"))
    
        # Dropping series without release year
        .dropna(subset=["release_year"])
        .dropna(subset=["end_year"])

        # Dropping duplicated rows
        .drop_duplicates(subset="series_title")

        .assign(duration= lambda df_: df_.end_year - df_.release_year)
    )

In [8]:
transformed_df = tweak_imdb_df(df)
print(transformed_df.shape)
transformed_df.sample(3)

(5952, 9)


,series_title,release_year,runtime,genre,rating,cast,synopsis,end_year,duration
8476,Shinui,2012,60,"Drama, Fantasy, History",8.0,"Lee Min-Ho, Kim Hee-seon, Yoon Kyun-Sang, Park...",When the queen-to-be of medieval Korea is badl...,2012,0
7626,Slugterra,2012,22,"Animation, Action, Adventure",7.0,"Lee Tockar, Sam Vincent, Andrew Francis, Shann...",Eli Shane is determined to be the greatest Slu...,2016,4
2523,Cybill,1995,30,Comedy,6.9,"Cybill Shepherd, Christine Baranski, Alicia Wi...","A struggling, middle-aged actress attempts to ...",1998,3


In [9]:
#create a new df with each genre in a new row and series title, rating, release year and end year
def explode_genres(df: pd.DataFrame) -> pd.DataFrame:
    return (
        df
        .assign(genre=df.genre.str.split(", "))
        .explode("genre")
    )

exploded_df = explode_genres(transformed_df)
print(exploded_df.shape)
exploded_df.sample(10)

(13629, 9)


,series_title,release_year,runtime,genre,rating,cast,synopsis,end_year,duration
4274,Hekimoglu,2019,120,Comedy,7.2,"Timuçin Esen, Okan Yalabik, Kaan Yildirim, Ayt...",Ates Hekimoglu is a successful doctor in his 4...,2021,2
11,The Walking Dead,2010,44,Horror,8.1,"Andrew Lincoln, Norman Reedus, Melissa McBride...",Sheriff Deputy Rick Grimes wakes up from a com...,2022,12
2507,Smash,2012,44,Drama,7.7,"Debra Messing, Jack Davenport, Katharine McPhe...",More drama occurs behind the scenes than on st...,2013,1
8238,Maniac,2015,25,Drama,7.1,"Espen Petrus Andersen Lervaag, Håkon Bast Moss...","Maniac is the story of Espen, a man in his thi...",2015,0
5878,Superjail!,2007,10,Crime,7.7,"David Wain, Teddy Cohn, Christopher McCulloch,...","In a surreal world, a large maximum security p...",2014,7
6139,Byker Grove,1989,30,Romance,6.1,"Daymon Britton, Billy Fane, Chris Beattie, Dec...",Byker Grove follows the lives and relationship...,2006,17
7538,Checkmate,1960,60,Mystery,7.7,"Anthony George, Doug McClure, Sebastian Cabot,...","Don Corey and Jed Sills operate Checkmate, Inc...",1962,2
8574,Inspector Max,2004,50,Comedy,5.2,"Fernando Luís, Fátima Belo, Sandra Celas, Rui ...",A police duo fight crime with the help of a do...,2019,15
561,A Million Little Things,2018,43,Romance,7.9,"David Giuntoli, Romany Malco, Allison Miller, ...",A group of friends becomes motivated to live f...,2023,5
9096,Bullets,2018,60,Thriller,6.5,"Krista Kosonen, Sibel Kekilli, Tommi Korpela, ...",Kosonen plays the exceptionally gifted underco...,2018,0


In [10]:
#add a decade column to the df based on the release year. The values should be in the format 1990-2000, 2000-2010, 2010-2020
def add_decade(df: pd.DataFrame) -> pd.DataFrame:
    return (
        df
        .assign(decade=lambda df_: df_.release_year.floordiv(10).mul(10).astype(str) + "-"+ (df_.release_year.floordiv(10).mul(10) + 10).astype(str))
    )

decade_df = add_decade(exploded_df)
print(decade_df.shape)
decade_df.sample(10)

(13629, 10)


,series_title,release_year,runtime,genre,rating,cast,synopsis,end_year,duration,decade
7660,Bakuman.,2010,24,Drama,8.0,"Dan Green, Jun'ichi Suwabe, Kevin T. Collins, ...","Moritaka Mashiro, a junior high school student...",2013,3,2010-2020
1547,Liar,2017,45,Drama,7.1,"Joanne Froggatt, Ioan Gruffudd, Zoë Tapper, Da...",British psychological thriller in which school...,2020,3,2010-2020
9013,Listen Up,2004,30,Comedy,5.7,"Jason Alexander, Wendy Makkena, Daniella Monet...","Tiny, O-shaped Tony Kleinman and his mate, for...",2005,1,2000-2010
1184,The New Addams Family,1998,30,Comedy,6.4,"Glenn Taranto, Ellie Harvie, Brody Smith, Nico...",An updated version of the classic series follo...,1999,1,1990-2000
2655,Clan,2012,50,Crime,7.9,"Barbara Sarafian, Kristine Van Pellicom, Ruth ...",Four sisters conspire to kill their obnoxious ...,2012,0,2010-2020
2392,Clarice,2021,60,Drama,6.6,"Rebecca Breeds, Michael Cudlitz, Jayne Atkinso...",A look at the personal story of FBI agent Clar...,2021,0,2020-2030
8656,Initial D: Fifth Stage,2012,<NA>,Action,8.0,"Takehito Koyasu, Shin'ichirô Miki",The final stage of Project D's expeditions is ...,2013,1,2010-2020
6463,Godzilla: The Series,1998,30,Action,6.8,"Ian Ziering, Malcolm Danare, Rino Romano, Char...",In this direct sequel to the 1998 Godzilla mov...,2001,3,1990-2000
7626,Slugterra,2012,22,Animation,7.0,"Lee Tockar, Sam Vincent, Andrew Francis, Shann...",Eli Shane is determined to be the greatest Slu...,2016,4,2010-2020
7092,Doraemon,1979,25,Animation,8.0,"Nobuyo Ôyama, Noriko Ohara, Kazuya Tatekabe, K...",Doraemon is a robotic cat that comes from the ...,2005,26,1970-1980


In [11]:
number_of_unique_genres = decade_df.genre.nunique()
print(f"Number of unique genres: {number_of_unique_genres}")

number_of_unique_release_years = decade_df.release_year.nunique()
print(f"Number of unique release years: {number_of_unique_release_years}")

number_of_unique_decades = decade_df.decade.nunique()
print(f"Number of unique release years: {number_of_unique_decades}")

#print uniqe genres
unique_genres = decade_df.genre.unique()
print(unique_genres)

Number of unique genres: 26
Number of unique release years: 79
Number of unique release years: 10
['Comedy' 'Drama' 'Western' 'Horror' 'Thriller' 'Action' 'Adventure'
 'Family' 'Mystery' 'Crime' 'Fantasy' 'Documentary' 'History' 'Romance'
 'Sci-Fi' 'Biography' 'Animation' 'Music' 'Sport' 'War' 'Short'
 'Reality-TV' 'Musical' 'Talk-Show' 'Game-Show' 'News']


In [12]:
columns_reodered = ['series_title','runtime','genre','rating', 'decade', 'release_year', 'end_year','duration', 'cast','synopsis']

In [13]:
tv_series = decade_df.reindex(columns = columns_reodered)
tv_series.to_csv('tv_series.csv', index=False)


In [14]:
print(tv_series.shape)
tv_series.sample(10)

(13629, 10)


,series_title,runtime,genre,rating,decade,release_year,end_year,duration,cast,synopsis
150,ER,44,Romance,7.9,1990-2000,1994,2009,15,"Anthony Edwards, George Clooney, Julianna Marg...","The lives, loves and losses of the doctors and..."
3410,Doc McStuffins,11,Family,6.6,2010-2020,2012,2022,10,"Lara Jill Miller, Robbie Rist, Loretta Devine,...","""Doc McStuffins"" is an imaginative animated se..."
2181,Above Suspicion,46,Drama,7.1,2000-2010,2009,2012,3,"Kelly Reilly, Ciarán Hinds, Shaun Dingwall, Am...",The murder of a young actress starts an invest...
1758,That Girl,30,Comedy,7.2,1960-1970,1966,1971,5,"Marlo Thomas, Ted Bessell, Lew Parker, Bernie ...",An aspiring actress moves from her hometown of...
369,Locke & Key,48,Fantasy,7.3,2020-2030,2020,2022,2,"Darby Stanchfield, Connor Jessup, Emilia Jones...",After their father is murdered under mysteriou...
1503,¿Quién Mató a Sara?,40,Drama,6.4,2020-2030,2021,2022,1,"Manolo Cardona, Ginés García Millán, Carolina ...",Hell-bent on exacting revenge and proving he w...
146,Sons of Anarchy,45,Crime,8.6,2000-2010,2008,2014,6,"Charlie Hunnam, Katey Sagal, Mark Boone Junior...",A biker struggles to balance being a father an...
6534,Het huis Anubis,12,Family,7.3,2000-2010,2006,2009,3,"Loek Beernink, Iris Hesseling, Walter Crommeli...",A group of boarding school students embark on ...
4449,Mighty Med,30,Sci-Fi,6.5,2010-2020,2013,2015,2,"Bradley Steven Perry, Jake Short, Paris Berelc...",When two comic book fanboys discover a secret ...
2010,Baptiste,60,Drama,7.3,2010-2020,2019,2021,2,"Tchéky Karyo, Anastasia Hille, Tom Hollander, ...",Mercurial retired police detective Julien Bapt...


In [15]:
#number of unique series_title with rating greater than 8

number_of_unique_series_title = tv_series[tv_series.rating > 8.5].series_title.nunique()
print(f"Number of unique series title with rating greater than 8: {number_of_unique_series_title}")

Number of unique series title with rating greater than 8: 275


In [16]:
#range of release year values
release_year_range = tv_series.release_year.agg([min, max])
print(f"Release year range: {release_year_range}")

#divide release year into decades
def release_year_to_decade(release_year: int) -> int:
    return release_year - release_year % 10

tv_series["decade"] = tv_series.release_year.apply(release_year_to_decade)
tv_series.sample(3)

#number of unique series_title in each decade
unique_series_title_per_decade = tv_series.groupby("decade").series_title.nunique()
print(unique_series_title_per_decade)

Release year range: min    1930
max    2023
Name: release_year, dtype: int64
decade
1930       1
1940       9
1950     127
1960     197
1970     305
1980     424
1990     705
2000    1046
2010    2402
2020     736
Name: series_title, dtype: int64


In [23]:




#for each decade, find the top 5 genres with the highest number of unique series_title and sort them in descending order , show count of series_title
top_genres_per_decade = (
    tv_series
    .query("rating > 5")
    .groupby(["decade", "genre"])
    .series_title
    .nunique()
    .groupby("decade", group_keys=False)
    .nlargest(10)
    .reset_index(name="series_count")
)
# print(top_genres_per_decade)

# drop decade 1930, 1940 and 2020 from top_genres_per_decade list
top_genres_per_decade = top_genres_per_decade[~top_genres_per_decade.decade.isin([1930, 1940, 2020])]
print(top_genres_per_decade)
unique_genres = top_genres_per_decade.genre.unique()
print(unique_genres)


#for each decade, find the top 5 genres with the highest percentage of unique series_title  and sort them in descending order , show count of series_title
# top_genres_per_decade_percentage = (
#     tv_series
#     .groupby(["decade", "genre"])
#     .series_title
#     .nunique()
#     .groupby("decade", group_keys=False)
#     .nlargest(5)
#     .div(tv_series.groupby("decade").series_title.count())
#     .mul(100)
#     .reset_index(name="percentage")
# )

# top_genres_per_decade = top_genres_per_decade_percentage[~top_genres_per_decade.decade.isin([1930, 1940, 2020])]
# print(top_genres_per_decade)
# unique_genres = top_genres_per_decade.genre.unique()
# print(unique_genres)


# #for each decade, find the top 5 genres with the highest percentage of unique series_title with rating > 8.5 out of all series_titel with rating > 8.5 and sort them in descending order , show count of series_title
# top_genres_per_decade_percentage = (
#     tv_series
#     .query("rating > 8.5")
#     .groupby(["decade", "genre"])
#     .series_title
#     .nunique()
#     .groupby("decade", group_keys=False)
#     .nlargest(5)
#     .div(tv_series.query("rating > 8.5").groupby("decade").series_title.count())
#     .mul(100)
#     .reset_index(name="percentage")
# )
    

# #drop decade 1930, 1940 and 2020 from top_genres_per_decade_percentage list
# top_genres_per_decade_percentage = top_genres_per_decade_percentage[~top_genres_per_decade_percentage.decade.isin([1930, 1940, 2020])]
# print(top_genres_per_decade_percentage)

# #print unique genres in top_genres_per_decade_percentage
# unique_genres = top_genres_per_decade_percentage.genre.unique()
# print(unique_genres)


    decade      genre  series_count
13    1950     Comedy            41
14    1950     Family            40
15    1950    Western            40
16    1950      Drama            39
17    1950  Adventure            21
..     ...        ...           ...
78    2010  Adventure           299
79    2010    Romance           292
80    2010    Mystery           243
81    2010    Fantasy           183
82    2010   Thriller           155

[70 rows x 3 columns]
['Comedy' 'Family' 'Western' 'Drama' 'Adventure' 'Crime' 'Action'
 'Mystery' 'Romance' 'Fantasy' 'Animation' 'Sci-Fi' 'Thriller']


In [ ]:
['Comedy', 'Family', 'Western', 'Drama', 'Adventure', 'Crime', 'Action', 'Mystery', 'Romance', 'Fantasy', 'Animation', 'Sci-Fi', 'Thriller', 'Others']